# Cucumber Steps Analysis Notebook

## Regex + Fuzzy Matcher of SDK Implemented Steps Against Actual Feature Steps

### Input - use the `make display-...` commands in each SDK
### Final outputs look like this [Google spreadsheet](https://docs.google.com/spreadsheets/d/1Szfvw6_OV0cTz-IEN9bTop5ZW2i472D-pmIHyPVJing/edit#gid=737750452)

In [ ]:
%pip install gherkin-official
%pip install fuzzywuzzy
%pip install python-Levenshtein

In [ ]:
from dataclasses import dataclass
from functools import reduce
from fuzzywuzzy import fuzz, process as fzp
from fuzzywuzzy import process
import gherkin
import gherkin.parser
import pandas as pd
from pathlib import Path

## Step 1: parse all Gherkin features

In [ ]:
def get_features_df():
    units = (Path.cwd().parent / "features" / "unit").glob("*.feature")
    integrations = (Path.cwd().parent / "features" / "integration").glob("*feature")
    dfu = pd.DataFrame(data=[{"test_type": "unit", "feature_path": line} for line in units])
    dfi = pd.DataFrame(data=[{"test_type": "integration", "feature_path": line} for line in integrations])
    feat_df = pd.concat([dfu, dfi])
    feat_df["feature"] = feat_df.feature_path.apply(lambda s: s.name)
    return feat_df.set_index("feature")

In [ ]:
feat_df = get_features_df()
feat_df

In [ ]:
@dataclass(eq=True, order=True, frozen=True)
class Step:
    keyword: str
    text: str

def parse_features(feature_path):
    parser = gherkin.parser.Parser()
    with open(feature_path) as f:
        feature_info = gherkin.token_scanner.TokenScanner(f.read())
    return parser.parse(feature_info)

def extract_all(key: str, d: dict, append_method="extend") -> list:
    extracted = []
    def extractor(_d):
        if key in _d:
            v = _d[key]
            if append_method == "extend":
                extracted.extend(v)
            else:
                extracted.append(v)
        for k, v in _d.items():
            if k != key and isinstance(v, dict):
                extractor(v)
            elif isinstance(v, list):
                for a in v:
                    if isinstance(a, dict):
                        extractor(a)
    extractor(d)
    return extracted

def uniq(s: list):
    return sorted(list(set(s)))

def extract_tags(feature_info: dict) -> list:
    return uniq([tag["name"] for tag in extract_all("tags", feature_info)])

def extract_steps(feature_info: dict) -> list:
    return uniq([Step(step["keyword"],step["text"]) for step in extract_all("steps", feature_info)])

#### Example: extract cucumber info for `c2c.feature`

In [ ]:
feature="c2c.feature"
feat_df = get_features_df()
eg_feature = feat_df[feat_df.index == feature].feature_path.iloc[0]

feature_info = parse_features(eg_feature)
tags = extract_tags(feature_info)
steps = extract_steps(feature_info)

tags, steps, feature_info
# feat_df

### All the Tags

In [ ]:
def append_tags(feat_df):
    def mapper(p):
        feature_info = parse_features(p)
        return extract_tags(feature_info)
    
    feat_df["tags"] = feat_df.feature_path.apply(mapper)

tags_df = get_features_df()
append_tags(tags_df)

tags_df[['test_type', 'tags']]

In [ ]:
all_tags = uniq(reduce(lambda l,x : l+x, tags_df.tags, []))
print(all_tags)

### All the Steps

In [ ]:
def get_step2feat(feat_df):
    def mapper(row):
        feature_info = parse_features(row.feature_path)
        return (row.feature, extract_steps(feature_info))
    
    feat_df = feat_df.reset_index()
    steps = feat_df.apply(mapper, axis=1)
    steps = pd.DataFrame(reduce(lambda xs, x: xs + [{"feature": x[0], "step": y} for y in x[1]], steps, []))
    steps["gwt"] = steps.step.apply(lambda step: step.keyword)
    steps["step"] = steps.step.apply(lambda step: step.text)

    feat_df = feat_df.set_index("feature")
    steps = steps.set_index("feature")

    return feat_df.join(steps)

def fill_step_templates(steps_df):
    subs = {
        r'"<[^>]*>"': '"hello"',
        r'<[^(][^>]*>': '42',
    }
    steps_df["filled_step"] = steps_df.step.replace(subs.keys(), subs.values(), regex=True)
    return steps_df

In [ ]:
all_steps = get_step2feat(get_features_df())
all_steps = fill_step_templates(all_steps)
all_steps

In [ ]:
filled_steps = all_steps.groupby(by="filled_step").count()
filled_steps = filled_steps[["step"]].rename(columns={"step": "count"}).sort_values(by="count", ascending=False)
filled_steps = filled_steps.reset_index()
filled_steps

In [ ]:
# filled_steps.to_clipboard()

## Step 2: parse the SDK-steps summaries into Pandas

## 2A) Java

### e.g., created in the Java SDK via `make display-all-java-steps`

```sh
find . 2>/dev/null | xargs grep "io.cucumber.java.en" 2>/dev/null | grep -v Binary | cut -d: -f1 | sort | uniq | xargs grep -E "@(Given|Then|When)"
```

#### `java_source2step`

In [ ]:
java_source2step = """./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @When("I build an application transaction with operation {string}, application-id {long}, sender {string}, approval-program {string}, clear-program {string}, global-bytes {long}, global-ints {long}, local-bytes {long}, local-ints {long}, app-args {string}, foreign-apps {string}, foreign-assets {string}, app-accounts {string}, fee {long}, first-valid {long}, last-valid {long}, genesis-hash {string}, extra-pages {long}")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @When("I build a keyreg transaction with sender {string}, nonparticipation {string}, vote first {int}, vote last {int}, key dilution {int}, vote public key {string}, selection public key {string}, and state proof public key {string}")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @Given("suggested transaction parameters fee {int}, flat-fee {string}, first-valid {int}, last-valid {int}, genesis-hash {string}, genesis-id {string}")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @When("I build a payment transaction with sender {string}, receiver {string}, amount {int}, close remainder to {string}")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @When("sign the transaction")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @Then("fee field is in txn")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @Then("fee field not in txn")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @Then("the base64 encoded signed transaction should equal {string}")
./src/test/java/com/algorand/algosdk/cucumber/shared/TransactionSteps.java:    @Then("the decoded transaction should equal the original")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Given("I build an application transaction with the transient account, the current application, suggested params, operation {string}, approval-program {string}, clear-program {string}, global-bytes {long}, global-ints {long}, local-bytes {long}, local-ints {long}, app-args {string}, foreign-apps {string}, foreign-assets {string}, app-accounts {string}, extra-pages {long}")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Given("I sign and submit the transaction, saving the txid. If there is an error it is {string}.")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Given("I wait for the transaction to be confirmed.")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Given("I remember the new application ID.")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Given("I reset the array of application IDs to remember.")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Given("I fund the current application's address with {int} microalgos.")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Then("I get the account address for the current application and see that it matches the app id's hash")
./src/test/java/com/algorand/algosdk/integration/Applications.java:    @Then("The transient account should have the created app {string} and total schema byte-slices {long} and uints {long}, the application {string} state contains key {string} with value {string}")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Given("I add the nonce {string}")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Given("suggested transaction parameters from the algod v2 client")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Given("an application id {int}")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Given("a new AtomicTransactionComposer")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I make a transaction signer for the transient account.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I create a transaction with signer with the current transaction.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I add the current transaction with signer to the composer.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("I clone the composer.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("I gather signatures with the composer.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I create the Method object from method signature {string}")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I create a new method arguments array.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I append the current transaction with signer to the method arguments array.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I append the encoded arguments {string} to the method arguments array.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("I execute the current transaction group with the composer.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("The app should have returned {string}.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("The app should have returned ABI types {string}.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("The {int}th atomic result for randomInt\\({int}) proves correct")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("The {int}th atomic result for randElement\\({string}) proves correct")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("The {int}th atomic result for {string} satisfies the regex {string}")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("I can dig the {int}th atomic result with path {string} and see the value {string}")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("I dig into the paths {string} of the resulting atomic transaction tree I see group ids and they are all the same")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @Then("The composer should have a status of {string}.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I add a method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I add a nonced method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I add a method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments, approval-program {string}, clear-program {string}, global-bytes {int}, global-ints {int}, local-bytes {int}, local-ints {int}, extra-pages {int}.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I add a method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments, approval-program {string}, clear-program {string}.")
./src/test/java/com/algorand/algosdk/integration/AtomicTxnComposer.java:    @When("I build the transaction group with the composer. If there is an error it is {string}.")
./src/test/java/com/algorand/algosdk/integration/Clients.java:    @Given("an algod v2 client connected to {string} port {int} with token {string}")
./src/test/java/com/algorand/algosdk/integration/Clients.java:    @Given("indexer client {int} at {string} port {int} with token {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create a wallet")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the wallet should exist")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get the wallet handle")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I can get the master derivation key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I rename the wallet")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I can still get the wallet information with the same handle")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I renew the wallet handle")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I release the wallet handle")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the wallet handle should not work")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("payment transaction parameters {int} {int} {int} {string} {string} {string} {int} {string} {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("key registration transaction parameters {int} {int} {int} {string} {string} {string} {int} {int} {int} {string} {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("mnemonic for private key {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create the payment transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create the key registration transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("default V2 key registration transaction {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("multisig addresses {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create the multisig payment transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I sign the multisig transaction with the private key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I sign the transaction with the private key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the signed transaction should equal the golden {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the multisig transaction should equal the golden {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the multisig address should equal the golden {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get versions with algod")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("v1 should be in the versions")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get versions with kmd")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get the status")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get status after this block")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I can get the block info")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I import the multisig")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the multisig should be in the wallet")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I export the multisig")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the multisig should equal the exported multisig")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I delete the multisig")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the multisig should not be in the wallet")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I generate a key using kmd")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I generate a key using kmd for rekeying and fund it")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the key should be in the wallet")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I delete the key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the key should not be in the wallet")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I generate a key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I import the key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get the private key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the private key should be equal to the exported private key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("a kmd client")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("an algod client")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("an algod v2 client")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("wallet information")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("default transaction with parameters {int} {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("default transaction with parameters {int} {string} and rekeying key")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("default multisig transaction with parameters {int} {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I send the transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I send the multisig transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the transaction should go through")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I can get the transaction by ID")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the transaction should not go through")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I sign the transaction with kmd")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the signed transaction should equal the kmd signed transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I sign the multisig transaction with kmd")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the multisig transaction should equal the kmd signed multisig transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I read a transaction {string} from file {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I write the transaction to file")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the transaction should still be the same")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I do my part")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the node should be healthy")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I get the ledger supply")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I get transactions by address and round")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I get transactions by address only")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I get transactions by address and date")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I get pending transactions")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get the suggested params")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get the suggested fee")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the fee in the suggested params should equal the suggested fee")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create a bid")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I encode and decode the bid")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I sign the bid")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the bid should still be the same")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I decode the address")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I encode the address")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the address should still be the same")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I convert the private key back to a mnemonic")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the mnemonic should still be the same as {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("mnemonic for master derivation key {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I convert the master derivation key back to a mnemonic")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create the flat fee payment transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("encoded multisig transaction {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I append a signature to the multisig transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("encoded multisig transactions {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I merge the multisig transactions")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I convert {long} microalgos to algos and back")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("it should still be the same amount of microalgos {long}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I get account information")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I can get account information")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get recent transactions, limited by {int} transactions")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("asset test fixture")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("default asset creation transaction with total issuance {int}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get the asset info")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the asset info should match the expected asset info")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create a no-managers asset reconfigure transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create an asset destroy transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I should be unable to get the asset info")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create a transaction transferring {int} assets from creator to a second account")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the creator should have {int} assets remaining")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I update the asset index")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I send the bogus kmd-signed transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I create a transaction for a second account, signalling asset acceptance")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("I send the kmd-signed transaction")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create a freeze transaction targeting the second account")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create a transaction transferring {int} assets from a second account to creator")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create an un-freeze transaction targeting the second account")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Given("default-frozen asset creation transaction with total issuance {int}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I create a transaction revoking {int} assets from a second account to creator")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I add a rekeyTo field with address {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I add a rekeyTo field with the private key algorand address")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I compile a teal program {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("base64 decoding the response is the same as the binary {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("it is compiled with {int} and {string} and {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I dryrun a {string} program {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I get execution result {string}")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @When("I compile a teal program {string} with mapping enabled")
./src/test/java/com/algorand/algosdk/integration/Stepdefs.java:    @Then("the resulting source map is the same as the json {string}")
./src/test/java/com/algorand/algosdk/integration/TransientAccount.java:    @Given("I create a new transient account and fund it with {long} microalgos.")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create the Method object from method signature {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I serialize the Method object into json")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create the Method object with name {string} first argument type {string} second argument type {string} and return type {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create the Method object with name {string} first argument name {string} first argument type {string} second argument name {string} second argument type {string} and return type {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create the Method object with name {string} method description {string} first argument type {string} first argument description {string} second argument type {string} second argument description {string} and return type {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create an Interface object from the Method object with name {string} and description {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I serialize the Interface object into json")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create a Contract object from the Method object with name {string} and description {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I set the Contract's appID to {int} for the network {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I serialize the Contract object into json")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @Then("the txn count should be {int}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @Then("the method selector should be {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @Then("the produced json should equal {string} loaded from {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @Then("the deserialized json should equal the original Method object")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @Then("the deserialized json should equal the original Interface object")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @Then("the deserialized json should equal the original Contract object")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I append to my Method objects list in the case of a non-empty signature {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create an Interface object from my Method objects list")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I get the method from the Interface by name {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I create a Contract object from my Method objects list")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @When("I get the method from the Contract by name {string}")
./src/test/java/com/algorand/algosdk/unit/ABIJson.java:    @Then("the produced method signature should equal {string}. If there is an error it begins with {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make a Pending Transaction Information against txid {string} with format {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make a Pending Transaction Information with max {long} and format {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make a Pending Transactions By Address call against account {string} and max {long} and format {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make a Status after Block call with round {long}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make an Account Information call against account {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make a Get Block call against block number {long} with format {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make a GetAssetByID call for assetID {long}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make a GetApplicationByID call for applicationID {long}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make an Account Application Information call against account {string} applicationID {int}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make an Account Asset Information call against account {string} assetID {int}")
./src/test/java/com/algorand/algosdk/unit/AlgodPaths.java:    @When("we make an Account Information call against account {string} with exclude {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Pending Transaction Information call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Pending Transactions Information call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Send Raw Transaction call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Pending Transactions By Address call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Node Status call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Ledger Supply call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Status After Block call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Account Information call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Get Block call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Suggested Transaction Parameters call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @When("we make any Dryrun call")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Pending Transaction Information response should have sender {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Pending Transactions Information response should contain an array of len {int} and element number {int} should have sender {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Pending Transactions By Address response should contain an array of len {int} and element number {int} should have sender {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Node Status response should have a last round of {int}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Ledger Supply response should have totalMoney {biginteger} onlineMoney {biginteger} on round {int}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Status After Block response should have a last round of {int}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Account Information response should have address {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Get Block response should have rewards pool {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Suggested Transaction Parameters response should have first round valid of {int}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Send Raw Transaction response should have txid {string}")
./src/test/java/com/algorand/algosdk/unit/AlgodResponses.java:    @Then("the parsed Dryrun Response should have global delta {string} with {int}")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @Given("an algod v2 client")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @Given("a new AtomicTransactionComposer")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @Given("an application id {int}")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I make a transaction signer for the signing account.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I create a transaction with signer with the current transaction.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I create a new method arguments array.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I append the current transaction with signer to the method arguments array.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I add the current transaction with signer to the composer.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I append the encoded arguments {string} to the method arguments array.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I add a method call with the signing account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I add a method call with the signing account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments, approval-program {string}, clear-program {string}, global-bytes {int}, global-ints {int}, local-bytes {int}, local-ints {int}, extra-pages {int}.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @When("I build the transaction group with the composer. If there is an error it is {string}.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @Then("The composer should have a status of {string}.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @Then("I gather signatures with the composer.")
./src/test/java/com/algorand/algosdk/unit/AtomicTxnComposer.java:    @Then("the base64 encoded signed transactions should equal {string}")
./src/test/java/com/algorand/algosdk/unit/Base.java:    @Given("a signing account with address {string} and mnemonic {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Asset Balances call against asset index {long} "
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Asset Transactions call against asset index {long} "
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Account Transactions call against account {string} "
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Block call against round {long}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Account by ID call against account {string} with round {long}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Asset by ID call against asset index {long}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Search Accounts call with assetID {long} "
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Search For Transactions call with account {string} "
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a SearchForAssets call with limit {long} "
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Asset Transactions call against asset index {long} with NotePrefix {string} TxType {string} SigType {string} txid {string} round {long} minRound {long} maxRound {long} limit {long} beforeTime {string} afterTime {string} currencyGreaterThan {long} currencyLessThan {long} address {string} addressRole {string} ExcluseCloseTo {string} RekeyTo {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Account Transactions call against account {string} with NotePrefix {string} TxType {string} SigType {string} txid {string} round {long} minRound {long} maxRound {long} limit {long} beforeTime {string} afterTime {string} currencyGreaterThan {long} currencyLessThan {long} assetIndex {long} rekeyTo {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Search Accounts call with assetID {long} limit {long} currencyGreaterThan {long} currencyLessThan {long} round {long} and authenticating address {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Search For Transactions call with account {string} NotePrefix {string} TxType {string} SigType {string} txid {string} round {long} minRound {long} maxRound {long} limit {long} beforeTime {string} afterTime {string} currencyGreaterThan {long} currencyLessThan {long} assetIndex {long} addressRole {string} ExcluseCloseTo {string} rekeyTo {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a SearchForApplications call with applicationID {long}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a LookupApplications call with applicationID {long}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a LookupApplicationLogsByID call with applicationID {long} limit {long} minRound {long} maxRound {long} nextToken {string} sender {string} and txID {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a LookupAccountAppLocalStates call with accountID {string} applicationID {int} includeAll {string} limit {int} next {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a LookupAccountAssets call with accountID {string} assetID {int} includeAll {string} limit {int} next {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Lookup Account by ID call against account {string} with exclude {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a LookupAccountCreatedApplications call with accountID {string} applicationID {int} includeAll {string} limit {int} next {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a LookupAccountCreatedAssets call with accountID {string} assetID {int} includeAll {string} limit {int} next {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a Search Accounts call with exclude {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerPaths.java:    @When("we make a SearchForApplications call with creator {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any LookupAssetBalances call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any LookupAssetTransactions call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any LookupAccountTransactions call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any LookupBlock call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any LookupAccountByID call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any LookupAssetByID call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any SearchAccounts call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any SearchForTransactions call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("we make any SearchForAssets call")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("expect error string to contain {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed LookupAssetBalances response should be valid on round {long}, "
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed LookupAssetTransactions response should be valid on round {long}, "
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed LookupAccountTransactions response should be valid on round {long}, "
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed LookupBlock response should have previous block hash {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed LookupAccountByID response should have address {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed LookupAssetByID response should have index {long}")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed SearchAccounts response should be valid on round {long} "
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed SearchForTransactions response should be valid on round {long} "
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @Then("the parsed SearchForAssets response should be valid on round {long} "
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("the parsed SearchAccounts response should be valid on round {int} and the array should be of len {int} and the element at index {int} should have authorizing address {string}")
./src/test/java/com/algorand/algosdk/unit/IndexerResponses.java:    @When("the parsed SearchForTransactions response should be valid on round {int} and the array should be of len {int} and the element at index {int} should have rekey-to {string}")
./src/test/java/com/algorand/algosdk/unit/PathsShared.java:    @Given("mock server recording request paths")
./src/test/java/com/algorand/algosdk/unit/PathsShared.java:    @Then("expect the path used to be {string}")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @Given("payment transaction parameters {int} {int} {int} {string} {string} {string} {int} {string} {string}")	
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @Given("mnemonic for private key {string}")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @When("I create the flat fee payment transaction")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @When("I add a rekeyTo field with address {string}")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @When("I sign the transaction with the private key")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @Then("the signed transaction should equal the golden {string}")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @When("I set the from address to {string}")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @Given("multisig addresses {string}")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @When("I create the multisig payment transaction")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @When("I create the multisig payment transaction with zero fee")
./src/test/java/com/algorand/algosdk/unit/Rekeying.java:    @When("I sign the multisig transaction with the private key")
./src/test/java/com/algorand/algosdk/unit/ResponsesShared.java:    @Given("mock http responses in {string} loaded from {string}")
./src/test/java/com/algorand/algosdk/unit/ResponsesShared.java:    @Given("mock http responses in {string} loaded from {string} with status {int}.")
./src/test/java/com/algorand/algosdk/unit/ResponsesShared.java:    @When("we make any {string} call to {string}.")
./src/test/java/com/algorand/algosdk/unit/ResponsesShared.java:    @Then("the parsed response should equal the mock response.")
./src/test/java/com/algorand/algosdk/unit/ResponsesShared.java:    @Given("a dryrun response file {string} and a transaction at index {string}")
./src/test/java/com/algorand/algosdk/unit/ResponsesShared.java:    @Then("calling app trace produces {string}")
./src/test/java/com/algorand/algosdk/unit/TealSign.java:    @Given("base64 encoded data to sign {string}")
./src/test/java/com/algorand/algosdk/unit/TealSign.java:    @Given("base64 encoded private key {string}")
./src/test/java/com/algorand/algosdk/unit/TealSign.java:    @Given("base64 encoded program {string}")
./src/test/java/com/algorand/algosdk/unit/TealSign.java:    @Given("program hash {string}")
./src/test/java/com/algorand/algosdk/unit/TealSign.java:    @When("I perform tealsign")
./src/test/java/com/algorand/algosdk/unit/TealSign.java:    @Then("the signature should be equal to {string}")
./src/test/java/com/algorand/algosdk/unit/TestSourceMap.java:    @Given("a source map json file {string}")
./src/test/java/com/algorand/algosdk/unit/TestSourceMap.java:    @Then("the string composed of pc:line number equals {string}")
./src/test/java/com/algorand/algosdk/unit/TestSourceMap.java:    @Then("getting the last pc associated with a line {string} equals {string}")
./src/test/java/com/algorand/algosdk/unit/TestSourceMap.java:    @Then("getting the line associated with a pc {string} equals {string}")"""

### `javasdk_df`

In [ ]:
java_source_and_step = [s2s.split(":    ") for s2s in java_source2step.split("\n")]
javasdk_df = pd.DataFrame(data=[{"source": line[0], "raw_step": line[1]} for line in java_source_and_step])
javasdk_df[["gwt","step"]] = javasdk_df.raw_step.str.extract(r'@([^(]*)."([^"]*)"')
javasdk_df

### Rexify - `javasdk_rex_df`

In [ ]:
subs = {
    ')': '\)',
    '{string}': '"([^"]*)"',
    '{int}': '(\d+)',
    '{long}': '(\d+)',
    '{biginteger}': '(\d+)',
}
javasdk_rex_df = javasdk_df.copy()
for k, v in subs.items():
    javasdk_rex_df["step"] = javasdk_rex_df.step.str.replace(k, v, regex=False)
javasdk_rex_df

In [ ]:
# javasdk_rex_df.to_clipboard()

## 2B) Python

### e.g., created in the Python SDK via `make display-all-python-steps`

```sh
find tests/steps -name "*.py" | xargs grep "behave" 2>/dev/null | cut -d: -f1 | sort | uniq | xargs awk "/@(given|step|then|when)/,/[)]/" | grep -E "(\".+\"|\'.+\')"
```

In [ ]:
py_steps = """    'we make an Account Information call against account "{account}" with exclude "{exclude:MaybeString}"'
@when('we make an Account Information call against account "{account}"')
    'we make a Lookup Account by ID call against account "{account}" with round {block}'
    'we make a Lookup Account by ID call against account "{account}" with exclude "{exclude:MaybeString}"'
@when("we make any LookupAccountByID call")
@then('the parsed LookupAccountByID response should have address "{address}"')
@when("we make any Account Information call")
    'the parsed Account Information response should have address "{address}"'
    'we make an Account Asset Information call against account "{account}" assetID {assetID}'
    'we make an Account Application Information call against account "{account}" applicationID {applicationID}'
    'we make a LookupAccountAssets call with accountID "{account}" assetID {asset_id} includeAll "{includeAll:MaybeBool}" limit {limit} next "{next:MaybeString}"'
    'we make a LookupAccountCreatedAssets call with accountID "{account}" assetID {asset_id} includeAll "{includeAll:MaybeBool}" limit {limit} next "{next:MaybeString}"'
    'we make a LookupAccountAppLocalStates call with accountID "{account}" applicationID {application_id} includeAll "{includeAll:MaybeBool}" limit {limit} next "{next:MaybeString}"'
    'we make a LookupAccountCreatedApplications call with accountID "{account}" applicationID {application_id} includeAll "{includeAll:MaybeBool}" limit {limit} next "{next:MaybeString}"'
    'the parsed LookupAssetBalances response should be valid on round {roundNum}, and contain an array of len {length} and element number {idx} should have address "{address}" amount {amount} and frozen state "{frozenState}"'
    "we make a Search Accounts call with assetID {index} limit {limit} currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} and round {block}"
    'we make a Search Accounts call with assetID {index} limit {limit} currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} round {block} and authenticating address "{authAddr:MaybeString}"'
@when('we make a Search Accounts call with exclude "{exclude:MaybeString}"')
@when("we make any SearchAccounts call")
    'the parsed SearchAccounts response should be valid on round {roundNum} and the array should be of len {length} and the element at index {index} should have address "{address}"'
    'the parsed SearchAccounts response should be valid on round {roundNum} and the array should be of len {length} and the element at index {index} should have authorizing address "{authAddr:MaybeString}"'
@given('a signing account with address "{address}" and mnemonic "{mnemonic}"')
    "I create a new transient account and fund it with {transient_fund_amount} microalgos."
    "I get the account address for the current application and see that it matches the app id's hash"
    "I fund the current application's address with {fund_amount} microalgos."
    'I sign and submit the transaction, saving the txid. If there is an error it is "{error_string:MaybeString}".'
@when("we make a GetApplicationByID call for applicationID {app_id}")
    'we make a LookupApplicationLogsByID call with applicationID {app_id} limit {limit} minRound {min_round} maxRound {max_round} nextToken "{next_token:MaybeString}" sender "{sender:MaybeString}" and txID "{txid:MaybeString}"'
@when("we make a SearchForApplications call with applicationID {app_id}")
@when('we make a SearchForApplications call with creator "{creator}"')
@when("we make a LookupApplications call with applicationID {app_id}")
@when("we make a SearchForApplications call with {application_id} and {round}")
@when("we make a LookupApplications call with {application_id} and {round}")
    'I build an application transaction with operation "{operation:MaybeString}", application-id {application_id}, sender "{sender:MaybeString}", approval-program "{approval_program:MaybeString}", clear-program "{clear_program:MaybeString}", global-bytes {global_bytes}, global-ints {global_ints}, local-bytes {local_bytes}, local-ints {local_ints}, app-args "{app_args:MaybeString}", foreign-apps "{foreign_apps:MaybeString}", foreign-assets "{foreign_assets:MaybeString}", app-accounts "{app_accounts:MaybeString}", fee {fee}, first-valid {first_valid}, last-valid {last_valid}, genesis-hash "{genesis_hash:MaybeString}", extra-pages {extra_pages}'
    'I build an application transaction with the transient account, the current application, suggested params, operation "{operation}", approval-program "{approval_program:MaybeString}", clear-program "{clear_program:MaybeString}", global-bytes {global_bytes}, global-ints {global_ints}, local-bytes {local_bytes}, local-ints {local_ints}, app-args "{app_args:MaybeString}", foreign-apps "{foreign_apps:MaybeString}", foreign-assets "{foreign_assets:MaybeString}", app-accounts "{app_accounts:MaybeString}", extra-pages {extra_pages}'
@given("I reset the array of application IDs to remember.")
@step("I remember the new application ID.")
@step("I wait for the transaction to be confirmed.")
@given("an application id {app_id}")
    'The transient account should have the created app "{app_created_bool_as_string:MaybeString}" and total schema byte-slices {byte_slices} and uints {uints}, the application "{application_state:MaybeString}" state contains key "{state_key:MaybeString}" with value "{state_value:MaybeString}"'
@given("a new AtomicTransactionComposer")
@step('I create the Method object from method signature "{method_signature}"')
@step("I make a transaction signer for the {account_type} account.")
@step("I create a transaction with signer with the current transaction.")
@when("I add the current transaction with signer to the composer.")
@step("I create a new method arguments array.")
    "I append the current transaction with signer to the method arguments array."
    'I append the encoded arguments "{method_args:MaybeString}" to the method arguments array.'
@given('I add the nonce "{nonce}"')
    'I add a method call with the {account_type} account, the current application, suggested params, on complete "{operation}", current transaction signer, current method arguments; any resulting exception has key "{exception_key}".'
    'I add a method call with the {account_type} account, the current application, suggested params, on complete "{operation}", current transaction signer, current method arguments.'
    'I add a method call with the {account_type} account, the current application, suggested params, on complete "{operation}", current transaction signer, current method arguments, approval-program "{approval_program_path:MaybeString}", clear-program "{clear_program_path:MaybeString}", global-bytes {global_bytes}, global-ints {global_ints}, local-bytes {local_bytes}, local-ints {local_ints}, extra-pages {extra_pages}.'
    'I add a method call with the {account_type} account, the current application, suggested params, on complete "{operation}", current transaction signer, current method arguments, approval-program "{approval_program_path:MaybeString}", clear-program "{clear_program_path:MaybeString}".'
    'I add a nonced method call with the {account_type} account, the current application, suggested params, on complete "{operation}", current transaction signer, current method arguments.'
    'I build the transaction group with the composer. If there is an error it is "{error_string:MaybeString}".'
@then('The composer should have a status of "{status}".')
@then("I gather signatures with the composer.")
@then("I clone the composer.")
@then("I execute the current transaction group with the composer.")
@then('The app should have returned "{returns:MaybeString}".')
@then('The app should have returned ABI types "{abiTypes:MaybeString}".')
@when("I serialize the Method object into json")
    'I create the Method object with name "{method_name}" method description "{method_desc}" first argument type "{first_arg_type}" first argument description "{first_arg_desc}" second argument type "{second_arg_type}" second argument description "{second_arg_desc}" and return type "{return_arg_type}"'
    'I create the Method object with name "{method_name}" first argument name "{first_arg_name}" first argument type "{first_arg_type}" second argument name "{second_arg_name}" second argument type "{second_arg_type}" and return type "{return_arg_type}"'
    'I create the Method object with name "{method_name}" first argument type "{first_arg_type}" second argument type "{second_arg_type}" and return type "{return_arg_type}"'
@then("the deserialized json should equal the original Method object")
@then("the txn count should be {txn_count}")
@then('the method selector should be "{method_selector}"')
    'I create an Interface object from the Method object with name "{interface_name}" and description "{description}"'
@when("I serialize the Interface object into json")
@then("the deserialized json should equal the original Interface object")
    'I create a Contract object from the Method object with name "{contract_name}" and description "{description}"'
@when('I set the Contract\'s appID to {app_id} for the network "{network_id}"')
@when("I serialize the Contract object into json")
@then("the deserialized json should equal the original Contract object")
    "The {result_index}th atomic result for randomInt({input}) proves correct"
    'The {result_index}th atomic result for randElement("{input}") proves correct'
    'The {result_index}th atomic result for "spin()" satisfies the regex "{regex}"'
    'I append to my Method objects list in the case of a non-empty signature "{method:MaybeString}"'
@when("I create an Interface object from my Method objects list")
@when("I create a Contract object from my Method objects list")
@when('I get the method from the Interface by name "{name}"')
@when('I get the method from the Contract by name "{name}"')
    'the produced method signature should equal "{methodsig}". If there is an error it begins with "{error:MaybeString}"'
@given("mock server recording request paths")
@given('mock http responses in "{jsonfiles}" loaded from "{directory}"')
    'mock http responses in "{filename}" loaded from "{directory}" with status {status}.'
@when('we make any "{client}" call to "{endpoint}".')
@then("the parsed response should equal the mock response.")
    'we make a Pending Transaction Information against txid "{txid}" with format "{response_format}"'
    'we make a Pending Transaction Information with max {max} and format "{response_format}"'
@when("we make any Pending Transactions Information call")
@when("we make any Pending Transaction Information call")
    'the parsed Pending Transaction Information response should have sender "{sender}"'
    'the parsed Pending Transactions Information response should contain an array of len {length} and element number {idx} should have sender "{sender}"'
    'we make a Pending Transactions By Address call against account "{account}" and max {max} and format "{response_format}"'
@when("we make any Pending Transactions By Address call")
    'the parsed Pending Transactions By Address response should contain an array of len {length} and element number {idx} should have sender "{sender}"'
@when("we make any Send Raw Transaction call")
@then('the parsed Send Raw Transaction response should have txid "{txid}"')
@when("we make any Node Status call")
@then("the parsed Node Status response should have a last round of {roundNum}")
@when("we make a Status after Block call with round {block}")
@when("we make any Status After Block call")
    "the parsed Status After Block response should have a last round of {roundNum}"
@when("we make any Ledger Supply call")
    "the parsed Ledger Supply response should have totalMoney {tot} onlineMoney {online} on round {roundNum}"
@when("we make a GetAssetByID call for assetID {asset_id}")
    'we make a Get Block call against block number {block} with format "{response_format}"'
@when("we make any Get Block call")
@then('the parsed Get Block response should have rewards pool "{pool}"')
    'we make a Lookup Asset Balances call against asset index {index} with limit {limit} afterAddress "{afterAddress:MaybeString}" currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan}'
@when("we make any LookupAssetBalances call")
    'we make a Lookup Asset Transactions call against asset index {index} with NotePrefix "{notePrefixB64:MaybeString}" TxType "{txType:MaybeString}" SigType "{sigType:MaybeString}" txid "{txid:MaybeString}" round {block} minRound {minRound} maxRound {maxRound} limit {limit} beforeTime "{beforeTime:MaybeString}" afterTime "{afterTime:MaybeString}" currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} address "{address:MaybeString}" addressRole "{addressRole:MaybeString}" ExcluseCloseTo "{excludeCloseTo:MaybeString}" RekeyTo "{rekeyTo:MaybeString}"'
    'we make a Lookup Asset Transactions call against asset index {index} with NotePrefix "{notePrefixB64:MaybeString}" TxType "{txType:MaybeString}" SigType "{sigType:MaybeString}" txid "{txid:MaybeString}" round {block} minRound {minRound} maxRound {maxRound} limit {limit} beforeTime "{beforeTime:MaybeString}" afterTime "{afterTime:MaybeString}" currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} address "{address:MaybeString}" addressRole "{addressRole:MaybeString}" ExcluseCloseTo "{excludeCloseTo:MaybeString}"'
@when("we make any LookupAssetTransactions call")
    'the parsed LookupAssetTransactions response should be valid on round {roundNum}, and contain an array of len {length} and element number {idx} should have sender "{sender}"'
    'we make a Lookup Account Transactions call against account "{account:MaybeString}" with NotePrefix "{notePrefixB64:MaybeString}" TxType "{txType:MaybeString}" SigType "{sigType:MaybeString}" txid "{txid:MaybeString}" round {block} minRound {minRound} maxRound {maxRound} limit {limit} beforeTime "{beforeTime:MaybeString}" afterTime "{afterTime:MaybeString}" currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} assetIndex {index} rekeyTo "{rekeyTo:MaybeString}"'
    'we make a Lookup Account Transactions call against account "{account:MaybeString}" with NotePrefix "{notePrefixB64:MaybeString}" TxType "{txType:MaybeString}" SigType "{sigType:MaybeString}" txid "{txid:MaybeString}" round {block} minRound {minRound} maxRound {maxRound} limit {limit} beforeTime "{beforeTime:MaybeString}" afterTime "{afterTime:MaybeString}" currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} assetIndex {index}'
@when("we make any LookupAccountTransactions call")
    'the parsed LookupAccountTransactions response should be valid on round {roundNum}, and contain an array of len {length} and element number {idx} should have sender "{sender}"'
@when("we make a Lookup Block call against round {block}")
@when("we make any LookupBlock call")
    'the parsed LookupBlock response should have previous block hash "{prevHash}"'
@when("we make a Lookup Asset by ID call against asset index {index}")
@when("we make any LookupAssetByID call")
@then("the parsed LookupAssetByID response should have index {index}")
@then("Every transaction works with asset-id {assetid}")
    'we make a Search For Transactions call with account "{account:MaybeString}" NotePrefix "{notePrefixB64:MaybeString}" TxType "{txType:MaybeString}" SigType "{sigType:MaybeString}" txid "{txid:MaybeString}" round {block} minRound {minRound} maxRound {maxRound} limit {limit} beforeTime "{beforeTime:MaybeString}" afterTime "{afterTime:MaybeString}" currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} assetIndex {index} addressRole "{addressRole:MaybeString}" ExcluseCloseTo "{excludeCloseTo:MaybeString}" rekeyTo "{rekeyTo:MaybeString}"'
    'we make a Search For Transactions call with account "{account:MaybeString}" NotePrefix "{notePrefixB64:MaybeString}" TxType "{txType:MaybeString}" SigType "{sigType:MaybeString}" txid "{txid:MaybeString}" round {block} minRound {minRound} maxRound {maxRound} limit {limit} beforeTime "{beforeTime:MaybeString}" afterTime "{afterTime:MaybeString}" currencyGreaterThan {currencyGreaterThan} currencyLessThan {currencyLessThan} assetIndex {index} addressRole "{addressRole:MaybeString}" ExcluseCloseTo "{excludeCloseTo:MaybeString}"'
@when("we make any SearchForTransactions call")
    'the parsed SearchForTransactions response should be valid on round {roundNum} and the array should be of len {length} and the element at index {index} should have sender "{sender}"'
    'the parsed SearchForTransactions response should be valid on round {roundNum} and the array should be of len {length} and the element at index {index} should have rekey-to "{rekeyTo:MaybeString}"'
@then('the parsed response should equal "{jsonfile}".')
    'we make a SearchForAssets call with limit {limit} creator "{creator:MaybeString}" name "{name:MaybeString}" unit "{unit:MaybeString}" index {index}'
@when("we make any SearchForAssets call")
    "the parsed SearchForAssets response should be valid on round {roundNum} and the array should be of len {length} and the element at index {index} should have asset index {assetIndex}"
@when("we make any Suggested Transaction Parameters call")
    "the parsed Suggested Transaction Parameters response should have first round valid of {roundNum}"
@then('expect the path used to be "{path}"')
@then('we expect the path used to be "{path}"')
@then('expect error string to contain "{err:MaybeString}"')
    'suggested transaction parameters fee {fee}, flat-fee "{flat_fee:MaybeBool}", first-valid {first_valid}, last-valid {last_valid}, genesis-hash "{genesis_hash}", genesis-id "{genesis_id}"'
    'I build a keyreg transaction with sender "{sender}", nonparticipation "{nonpart:MaybeBool}", vote first {vote_first}, vote last {vote_last}, key dilution {key_dilution}, vote public key "{vote_pk:MaybeString}", selection public key "{selection_pk:MaybeString}", and state proof public key "{state_proof_pk:MaybeString}"'
@given("suggested transaction parameters from the algod v2 client")
    'I build a payment transaction with sender "{sender:MaybeString}", receiver "{receiver:MaybeString}", amount {amount}, close remainder to "{close_remainder_to:MaybeString}"'
@when("sign the transaction")
@then('the base64 encoded signed transactions should equal "{goldens}"')
@then('the base64 encoded signed transaction should equal "{golden}"')
@then("the decoded transaction should equal the original")
    'an algod v2 client connected to "{host}" port {port} with token "{token}"'
@given("an algod v2 client")
@when('I compile a teal program "{program}"')
    'it is compiled with {status} and "{result:MaybeString}" and "{hash:MaybeString}"'
    'base64 decoding the response is the same as the binary "{binary:MaybeString}"'
@when('I dryrun a "{kind}" program "{program}"')
@then('I get execution result "{result}"')
@when("we make any Dryrun call")
    'the parsed Dryrun Response should have global delta "{creator}" with {action}'
@given('dryrun test case with "{program}" of type "{kind}"')
@then('status assert of "{status}" is succeed')
@then('global delta assert with "{key}", "{value}" and {action} is succeed')
@then('global delta assert with "{key}", "{value}" and {action} is failed')
    'local delta assert for "{account}" of accounts {index} with "{key}", "{value}" and {action} is succeed'
    'the produced json should equal "{json_path}" loaded from "{json_directory}"'
    'a dryrun response file "{dryrun_response_file}" and a transaction at index "{txn_id}"'
@then('calling app trace produces "{app_trace_file}"')
    'I dig into the paths "{paths}" of the resulting atomic transaction tree I see group ids and they are all the same'
    'I can dig the {i}th atomic result with path "{path}" and see the value "{field}"'
@given('a source map json file "{sourcemap_file}"')
@then('the string composed of pc:line number equals "{pc_to_line}"')
@then('getting the line associated with a pc "{pc}" equals "{line}"')
@then('getting the last pc associated with a line "{line}" equals "{pc}"')
@when('I compile a teal program "{teal}" with mapping enabled')
@then('the resulting source map is the same as the json "{sourcemap}"')
@when("I create a wallet")
@then("the wallet should exist")
@when("I get the wallet handle")
@then("I can get the master derivation key")
@when("I rename the wallet")
@then("I can still get the wallet information with the same handle")
@when("I renew the wallet handle")
@when("I release the wallet handle")
@then("the wallet handle should not work")
    'payment transaction parameters {fee} {fv} {lv} "{gh}" "{to}" "{close}" {amt} "{gen}" "{note}"'
@given('mnemonic for private key "{mn}"')
@when("I create the payment transaction")
@given('multisig addresses "{addresses}"')
@when("I create the multisig payment transaction")
@when("I create the multisig payment transaction with zero fee")
@when("I sign the multisig transaction with the private key")
@when("I sign the transaction with the private key")
@then('the signed transaction should equal the golden "{golden}"')
@then('the multisig address should equal the golden "{golden}"')
@then('the multisig transaction should equal the golden "{golden}"')
@when("I get versions with algod")
@then("v1 should be in the versions")
@when("I get versions with kmd")
@when("I get the status")
@when("I get status after this block")
@then("I can get the block info")
@when("I import the multisig")
@then("the multisig should be in the wallet")
@when("I export the multisig")
@then("the multisig should equal the exported multisig")
@when("I delete the multisig")
@then("the multisig should not be in the wallet")
@when("I generate a key using kmd")
@when("I generate a key using kmd for rekeying and fund it")
@then("the key should be in the wallet")
@when("I delete the key")
@then("the key should not be in the wallet")
@when("I generate a key")
@when("I import the key")
@then("the private key should be equal to the exported private key")
@given("a kmd client")
@given("an algod client")
@given("wallet information")
@given('default transaction with parameters {amt} "{note}"')
@given('default transaction with parameters {amt} "{note}" and rekeying key')
@given('default multisig transaction with parameters {amt} "{note}"')
@when("I get the private key")
@when("I send the transaction")
@when("I send the kmd-signed transaction")
@when("I send the bogus kmd-signed transaction")
@when("I send the multisig transaction")
@then("the transaction should go through")
@then("the transaction should not go through")
@when("I sign the transaction with kmd")
@then("the signed transaction should equal the kmd signed transaction")
@when("I sign the multisig transaction with kmd")
    "the multisig transaction should equal the kmd signed multisig transaction"
@when('I read a transaction "{txn}" from file "{num}"')
@when("I write the transaction to file")
@then("the transaction should still be the same")
@then("I do my part")
@then("I get the ledger supply")
@then("the node should be healthy")
@when("I get the suggested params")
@when("I get the suggested fee")
@then("the fee in the suggested params should equal the suggested fee")
@when("I create a bid")
@when("I encode and decode the bid")
@then("the bid should still be the same")
@when("I sign the bid")
@when("I decode the address")
@when("I encode the address")
@then("the address should still be the same")
@when("I convert the private key back to a mnemonic")
@then('the mnemonic should still be the same as "{mn}"')
@given('mnemonic for master derivation key "{mn}"')
@when("I convert the master derivation key back to a mnemonic")
@when("I create the flat fee payment transaction")
@given('encoded multisig transaction "{mtx}"')
@when("I append a signature to the multisig transaction")
@given('encoded multisig transactions "{msigtxns}"')
@when("I merge the multisig transactions")
@when("I convert {microalgos} microalgos to algos and back")
@then("it should still be the same amount of microalgos {microalgos}")
@then("I get transactions by address and round")
@then("I get pending transactions")
@then("I get account information")
@then("I can get account information")
    'key registration transaction parameters {fee} {fv} {lv} "{gh}" "{votekey}" "{selkey}" {votefst} {votelst} {votekd} "{gen}" "{note}"'
@when("I create the key registration transaction")
@given("default V2 key registration transaction {type}")
@given("default asset creation transaction with total issuance {total}")
@given("default-frozen asset creation transaction with total issuance {total}")
@given("asset test fixture")
@when("I update the asset index")
@when("I get the asset info")
@then("the asset info should match the expected asset info")
@when("I create an asset destroy transaction")
@then("I should be unable to get the asset info")
@when("I create a no-managers asset reconfigure transaction")
    "I create a transaction for a second account, signalling asset acceptance"
    "I create a transaction transferring {amount} assets from creator to a second account"
    "I create a transaction transferring {amount} assets from a second account to creator"
@then("the creator should have {exp_balance} assets remaining")
@when("I create a freeze transaction targeting the second account")
@when("I create an un-freeze transaction targeting the second account")
    "I create a transaction revoking {amount} assets from a second account to creator"
@given("I sign the transaction with the private key")
@given("I send the transaction")
@when('mnemonic for private key "{mn}"')
@when('I set the from address to "{from_addr}"')
@when("I add a rekeyTo field with the private key algorand address")
@when('I add a rekeyTo field with address "{rekey}"')
@given('base64 encoded data to sign "{data_enc}"')
@given('program hash "{contract_addr}"')
@when("I perform tealsign")
@then('the signature should be equal to "{sig_enc}"')
@given('base64 encoded program "{program_enc}"')
@given('base64 encoded private key "{sk_enc}"')
@then("fee field is in txn")
@then("fee field not in txn")"""


### `pysdk_df` - Unprocessed Steps

In [ ]:
pysdk_df = pd.DataFrame(data=[{"raw_step": line} for line in py_steps.split("\n")])
pysdk_df[["1", "2"]] = pysdk_df.raw_step.str.extract(r'@(given|step|then|when).(.*).')
pysdk_df["step"] = pysdk_df.apply(lambda row: (row["raw_step"][4:] if pd.isnull(row["2"]) else row["2"])[1:-1], axis=1)
pysdk_df = pysdk_df.drop(["1", "2"], axis=1)
pysdk_df

### Rexify - `pysdk_rex_df`

In [ ]:
subs = {
    r'"\{[^}]*\}"': '"([^"]*)"',
    r'\{[^(][^}]*\}': '([0-9]+)',
}
pysdk_rex_df = pysdk_df.copy()
pysdk_rex_df["step"] = pysdk_rex_df.step.replace(subs.keys(), subs.values(), regex=True)
pysdk_rex_df

In [ ]:
# pysdk_rex_df.to_clipboard()

## 2C) Go

### e.g., created in the Go SDK via `make display-all-go-steps`

```sh
find test -name "*.go" | xargs grep "github.com/cucumber/godog" 2>/dev/null | cut -d: -f1 | sort | uniq | xargs grep -Eo "Step[(].[^\`]+" | awk '{sub(/:Step\(./,":")} 1' | sed -E 's/", [a-zA-Z0-9]+\)//g'
```

In [ ]:
go_source2step ="""test/algodclientv2_test.go:^mock http responses in "([^"]*)" loaded from "([^"]*)"$
test/algodclientv2_test.go:^expect error string to contain "([^"]*)"$
test/algodclientv2_test.go:^we make any Pending Transaction Information call$
test/algodclientv2_test.go:^the parsed Pending Transaction Information response should have sender "([^"]*)"$
test/algodclientv2_test.go:^we make any Pending Transactions Information call$
test/algodclientv2_test.go:^the parsed Pending Transactions Information response should have sender "([^"]*)"$
test/algodclientv2_test.go:^we make any Send Raw Transaction call$
test/algodclientv2_test.go:^the parsed Send Raw Transaction response should have txid "([^"]*)"$
test/algodclientv2_test.go:^we make any Pending Transactions By Address call$
test/algodclientv2_test.go:^the parsed Pending Transactions By Address response should contain an array of len (\d+) and element number (\d+) should have sender "([^"]*)"$
test/algodclientv2_test.go:^we make any Node Status call$
test/algodclientv2_test.go:^the parsed Node Status response should have a last round of (\d+)$
test/algodclientv2_test.go:^we make any Ledger Supply call$
test/algodclientv2_test.go:^the parsed Ledger Supply response should have totalMoney (\d+) onlineMoney (\d+) on round (\d+)$
test/algodclientv2_test.go:^we make any Status After Block call$
test/algodclientv2_test.go:^the parsed Status After Block response should have a last round of (\d+)$
test/algodclientv2_test.go:^we make any Account Information call$
test/algodclientv2_test.go:^the parsed Account Information response should have address "([^"]*)"$
test/algodclientv2_test.go:^we make any Get Block call$
test/algodclientv2_test.go:^the parsed Get Block response should have rewards pool "([^"]*)"$
test/algodclientv2_test.go:^we make any Suggested Transaction Parameters call$
test/algodclientv2_test.go:^the parsed Suggested Transaction Parameters response should have first round valid of (\d+)$
test/algodclientv2_test.go:^expect the path used to be "([^"]*)"$
test/algodclientv2_test.go:^we make a Pending Transaction Information against txid "([^"]*)" with max (\d+)$
test/algodclientv2_test.go:^we make a Pending Transactions By Address call against account "([^"]*)" and max (\d+)$
test/algodclientv2_test.go:^we make a Status after Block call with round (\d+)$
test/algodclientv2_test.go:^we make an Account Information call against account "([^"]*)"$
test/algodclientv2_test.go:^we make a Get Block call against block number (\d+)$
test/algodclientv2_test.go:^the parsed Pending Transactions Information response should contain an array of len (\d+) and element number (\d+) should have sender "([^"]*)"$
test/algodclientv2_test.go:^we make a Pending Transaction Information against txid "([^"]*)" with format "([^"]*)"$
test/algodclientv2_test.go:^we make a Pending Transaction Information with max (\d+) and format "([^"]*)"$
test/algodclientv2_test.go:^we make a Pending Transactions By Address call against account "([^"]*)" and max (\d+) and format "([^"]*)"$
test/algodclientv2_test.go:^we make a Get Block call against block number (\d+) with format "([^"]*)"$
test/algodclientv2_test.go:^we make any Dryrun call$
test/algodclientv2_test.go:^the parsed Dryrun Response should have global delta "([^"]*)" with (\d+)$
test/algodclientv2_test.go:^we make an Account Information call against account "([^"]*)" with exclude "([^"]*)"$
test/algodclientv2_test.go:^we make an Account Asset Information call against account "([^"]*)" assetID (\d+)$
test/algodclientv2_test.go:^we make an Account Application Information call against account "([^"]*)" applicationID (\d+)$
test/applications_integration_test.go:^an algod v(\d+) client connected to "([^"]*)" port (\d+) with token "([^"]*)"$
test/applications_integration_test.go:^I create a new transient account and fund it with (\d+) microalgos\.$
test/applications_integration_test.go:^I build an application transaction with the transient account, the current application, suggested params, operation "([^"]*)", approval-program "([^"]*)", clear-program "([^"]*)", global-bytes (\d+), global-ints (\d+), local-bytes (\d+), local-ints (\d+), app-args "([^"]*)", foreign-apps "([^"]*)", foreign-assets "([^"]*)", app-accounts "([^"]*)", extra-pages (\d+)$
test/applications_integration_test.go:^I sign and submit the transaction, saving the txid\. If there is an error it is "([^"]*)"\.$
test/applications_integration_test.go:^I wait for the transaction to be confirmed\.$
test/applications_integration_test.go:^I remember the new application ID\.$
test/applications_integration_test.go:^I reset the array of application IDs to remember\.$
test/applications_integration_test.go:^I get the account address for the current application and see that it matches the app id\'s hash$
test/applications_integration_test.go:^The transient account should have the created app "([^"]*)" and total schema byte-slices (\d+) and uints (\d+), the application "([^"]*)" state contains key "([^"]*)" with value "([^"]*)"$
test/applications_integration_test.go:^suggested transaction parameters from the algod v2 client$
test/applications_integration_test.go:^I add the current transaction with signer to the composer\.$
test/applications_integration_test.go:^I clone the composer\.$
test/applications_integration_test.go:^I execute the current transaction group with the composer\.$
test/applications_integration_test.go:^The app should have returned "([^"]*)"\.$
test/applications_integration_test.go:^The app should have returned ABI types "([^"]*)"\.$
test/applications_integration_test.go:^I fund the current application\'s address with (\d+) microalgos\.$
test/applications_integration_test.go:^I can dig the (\d+)th atomic result with path "([^"]*)" and see the value "([^"]*)"$
test/applications_integration_test.go:^I dig into the paths "([^"]*)" of the resulting atomic transaction tree I see group ids and they are all the same$
test/applications_integration_test.go:^The (\d+)th atomic result for "([^"]*)" satisfies the regex "([^"]*)"$
test/applications_integration_test.go:^The (\d+)th atomic result for randomInt\((\d+)\) proves correct$
test/applications_integration_test.go:^The (\d+)th atomic result for randElement\("([^"]*)"\) proves correct$
test/applications_unit_test.go:^fee field is in txn$
test/applications_unit_test.go:^fee field not in txn$
test/applications_unit_test.go:^we make a GetAssetByID call for assetID (\d+)$
test/applications_unit_test.go:^we make a GetApplicationByID call for applicationID (\d+)$
test/applications_unit_test.go:^we make a SearchForApplications call with applicationID (\d+)$
test/applications_unit_test.go:^we make a LookupApplications call with applicationID (\d+)$
test/indexer_unit_test.go:^we make any LookupAssetBalances call$
test/indexer_unit_test.go:^the parsed LookupAssetBalances response should be valid on round (\d+), and contain an array of len (\d+) and element number (\d+) should have address "([^"]*)" amount (\d+) and frozen state "([^"]*)"$
test/indexer_unit_test.go:^we make any LookupAssetTransactions call$
test/indexer_unit_test.go:^the parsed LookupAssetTransactions response should be valid on round (\d+), and contain an array of len (\d+) and element number (\d+) should have sender "([^"]*)"$
test/indexer_unit_test.go:^we make any LookupAccountTransactions call$
test/indexer_unit_test.go:^the parsed LookupAccountTransactions response should be valid on round (\d+), and contain an array of len (\d+) and element number (\d+) should have sender "([^"]*)"$
test/indexer_unit_test.go:^we make any LookupBlock call$
test/indexer_unit_test.go:^the parsed LookupBlock response should have previous block hash "([^"]*)"$
test/indexer_unit_test.go:^we make any LookupAccountByID call$
test/indexer_unit_test.go:^the parsed LookupAccountByID response should have address "([^"]*)"$
test/indexer_unit_test.go:^we make any LookupAssetByID call$
test/indexer_unit_test.go:^the parsed LookupAssetByID response should have index (\d+)$
test/indexer_unit_test.go:^we make any SearchAccounts call$
test/indexer_unit_test.go:^the parsed SearchAccounts response should be valid on round (\d+) and the array should be of len (\d+) and the element at index (\d+) should have address "([^"]*)"$
test/indexer_unit_test.go:^we make any SearchForTransactions call$
test/indexer_unit_test.go:^the parsed SearchForTransactions response should be valid on round (\d+) and the array should be of len (\d+) and the element at index (\d+) should have sender "([^"]*)"$
test/indexer_unit_test.go:^we make any SearchForAssets call$
test/indexer_unit_test.go:^the parsed SearchForAssets response should be valid on round (\d+) and the array should be of len (\d+) and the element at index (\d+) should have asset index (\d+)$
test/indexer_unit_test.go:^we make a Lookup Asset Balances call against asset index (\d+) with limit (\d+) afterAddress "([^"]*)" currencyGreaterThan (\d+) currencyLessThan (\d+)$
test/indexer_unit_test.go:^we make a Lookup Asset Transactions call against asset index (\d+) with NotePrefix "([^"]*)" TxType "([^"]*)" SigType "([^"]*)" txid "([^"]*)" round (\d+) minRound (\d+) maxRound (\d+) limit (\d+) beforeTime "([^"]*)" afterTime "([^"]*)" currencyGreaterThan (\d+) currencyLessThan (\d+) address "([^"]*)" addressRole "([^"]*)" ExcluseCloseTo "([^"]*)"$
test/indexer_unit_test.go:^we make a Search Accounts call with assetID (\d+) limit (\d+) currencyGreaterThan (\d+) currencyLessThan (\d+) and round (\d+)$
test/indexer_unit_test.go:^we make a Lookup Account Transactions call against account "([^"]*)" with NotePrefix "([^"]*)" TxType "([^"]*)" SigType "([^"]*)" txid "([^"]*)" round (\d+) minRound (\d+) maxRound (\d+) limit (\d+) beforeTime "([^"]*)" afterTime "([^"]*)" currencyGreaterThan (\d+) currencyLessThan (\d+) assetIndex (\d+)$
test/indexer_unit_test.go:^we make a Lookup Block call against round (\d+)$
test/indexer_unit_test.go:^we make a Lookup Account by ID call against account "([^"]*)" with round (\d+)$
test/indexer_unit_test.go:^we make a Lookup Asset by ID call against asset index (\d+)$
test/indexer_unit_test.go:^we make a Search For Transactions call with account "([^"]*)" NotePrefix "([^"]*)" TxType "([^"]*)" SigType "([^"]*)" txid "([^"]*)" round (\d+) minRound (\d+) maxRound (\d+) limit (\d+) beforeTime (\d+) afterTime (\d+) currencyGreaterThan (\d+) currencyLessThan (\d+) assetIndex (\d+) addressRole "([^"]*)" ExcluseCloseTo "([^"]*)"$
test/indexer_unit_test.go:^we make a SearchForAssets call with limit (\d+) creator "([^"]*)" name "([^"]*)" unit "([^"]*)" index (\d+) and afterAsset (\d+)$
test/indexer_unit_test.go:^mock server recording request paths
test/indexer_unit_test.go:^we make a Search For Transactions call with account "([^"]*)" NotePrefix "([^"]*)" TxType "([^"]*)" SigType "([^"]*)" txid "([^"]*)" round (\d+) minRound (\d+) maxRound (\d+) limit (\d+) beforeTime "([^"]*)" afterTime "([^"]*)" currencyGreaterThan (\d+) currencyLessThan (\d+) assetIndex (\d+) addressRole "([^"]*)" ExcluseCloseTo "([^"]*)"$
test/indexer_unit_test.go:^we make a SearchForAssets call with limit (\d+) creator "([^"]*)" name "([^"]*)" unit "([^"]*)" index (\d+)$
test/indexer_unit_test.go:^we make a Lookup Account Transactions call against account "([^"]*)" with NotePrefix "([^"]*)" TxType "([^"]*)" SigType "([^"]*)" txid "([^"]*)" round (\d+) minRound (\d+) maxRound (\d+) limit (\d+) beforeTime "([^"]*)" afterTime "([^"]*)" currencyGreaterThan (\d+) currencyLessThan (\d+) assetIndex (\d+) rekeyTo "([^"]*)"$
test/indexer_unit_test.go:^we make a Search Accounts call with assetID (\d+) limit (\d+) currencyGreaterThan (\d+) currencyLessThan (\d+) round (\d+) and authenticating address "([^"]*)"$
test/indexer_unit_test.go:^we make a Search For Transactions call with account "([^"]*)" NotePrefix "([^"]*)" TxType "([^"]*)" SigType "([^"]*)" txid "([^"]*)" round (\d+) minRound (\d+) maxRound (\d+) limit (\d+) beforeTime "([^"]*)" afterTime "([^"]*)" currencyGreaterThan (\d+) currencyLessThan (\d+) assetIndex (\d+) addressRole "([^"]*)" ExcluseCloseTo "([^"]*)" rekeyTo "([^"]*)"$
test/indexer_unit_test.go:^the parsed SearchAccounts response should be valid on round (\d+) and the array should be of len (\d+) and the element at index (\d+) should have authorizing address "([^"]*)"$
test/indexer_unit_test.go:^the parsed SearchForTransactions response should be valid on round (\d+) and the array should be of len (\d+) and the element at index (\d+) should have rekey-to "([^"]*)"$
test/indexer_unit_test.go:^we make a Lookup Asset Transactions call against asset index (\d+) with NotePrefix "([^"]*)" TxType "([^"]*)" SigType "([^"]*)" txid "([^"]*)" round (\d+) minRound (\d+) maxRound (\d+) limit (\d+) beforeTime "([^"]*)" afterTime "([^"]*)" currencyGreaterThan (\d+) currencyLessThan (\d+) address "([^"]*)" addressRole "([^"]*)" ExcluseCloseTo "([^"]*)" RekeyTo "([^"]*)"$
test/indexer_unit_test.go:^we make a LookupApplicationLogsByID call with applicationID (\d+) limit (\d+) minRound (\d+) maxRound (\d+) nextToken "([^"]*)" sender "([^"]*)" and txID "([^"]*)"$
test/indexer_unit_test.go:^we make a LookupAccountAssets call with accountID "([^"]*)" assetID (\d+) includeAll "([^"]*)" limit (\d+) next "([^"]*)"$
test/indexer_unit_test.go:^we make a LookupAccountCreatedAssets call with accountID "([^"]*)" assetID (\d+) includeAll "([^"]*)" limit (\d+) next "([^"]*)"$
test/indexer_unit_test.go:^we make a LookupAccountAppLocalStates call with accountID "([^"]*)" applicationID (\d+) includeAll "([^"]*)" limit (\d+) next "([^"]*)"$
test/indexer_unit_test.go:^we make a LookupAccountCreatedApplications call with accountID "([^"]*)" applicationID (\d+) includeAll "([^"]*)" limit (\d+) next "([^"]*)"$
test/indexer_unit_test.go:^we make a Search Accounts call with exclude "([^"]*)"$
test/indexer_unit_test.go:^we make a Lookup Account by ID call against account "([^"]*)" with exclude "([^"]*)"$
test/indexer_unit_test.go:^we make a SearchForApplications call with creator "([^"]*)"$
test/responses_unit_test.go:^mock http responses in "([^"]*)" loaded from "([^"]*)" with status (\d+)\.$
test/responses_unit_test.go:^we make any "([^"]*)" call to "([^"]*)"\.$
test/responses_unit_test.go:^the parsed response should equal the mock response\.$
test/steps_test.go:I create a wallet
test/steps_test.go:the wallet should exist
test/steps_test.go:I get the wallet handle
test/steps_test.go:I can get the master derivation key
test/steps_test.go:I rename the wallet
test/steps_test.go:I can still get the wallet information with the same handle
test/steps_test.go:I renew the wallet handle
test/steps_test.go:I release the wallet handle
test/steps_test.go:the wallet handle should not work
test/steps_test.go:payment transaction parameters (\d+) (\d+) (\d+) "([^"]*)" "([^"]*)" "([^"]*)" (\d+) "([^"]*)" "([^"]*)"
test/steps_test.go:mnemonic for private key "([^"]*)"
test/steps_test.go:I create the payment transaction
test/steps_test.go:multisig addresses "([^"]*)"
test/steps_test.go:I create the multisig payment transaction$
test/steps_test.go:I create the multisig payment transaction with zero fee
test/steps_test.go:I sign the multisig transaction with the private key
test/steps_test.go:I sign the transaction with the private key
test/steps_test.go:^I add a rekeyTo field with address "([^"]*)"$
test/steps_test.go:^I add a rekeyTo field with the private key algorand address$
test/steps_test.go:^I set the from address to "([^"]*)"$
test/steps_test.go:the signed transaction should equal the golden "([^"]*)"
test/steps_test.go:the multisig transaction should equal the golden "([^"]*)"
test/steps_test.go:the multisig address should equal the golden "([^"]*)"
test/steps_test.go:I get versions with algod
test/steps_test.go:v1 should be in the versions
test/steps_test.go:I get versions with kmd
test/steps_test.go:I get the status
test/steps_test.go:^I get status after this block
test/steps_test.go:I can get the block info
test/steps_test.go:I import the multisig
test/steps_test.go:the multisig should be in the wallet
test/steps_test.go:I export the multisig
test/steps_test.go:the multisig should equal the exported multisig
test/steps_test.go:I delete the multisig
test/steps_test.go:the multisig should not be in the wallet
test/steps_test.go:^I generate a key using kmd for rekeying and fund it$
test/steps_test.go:^I generate a key using kmd$
test/steps_test.go:the key should be in the wallet
test/steps_test.go:I delete the key
test/steps_test.go:the key should not be in the wallet
test/steps_test.go:I generate a key
test/steps_test.go:I import the key
test/steps_test.go:the private key should be equal to the exported private key
test/steps_test.go:a kmd client
test/steps_test.go:an algod client
test/steps_test.go:wallet information
test/steps_test.go:default transaction with parameters (\d+) "([^"]*)"
test/steps_test.go:default multisig transaction with parameters (\d+) "([^"]*)"
test/steps_test.go:I get the private key
test/steps_test.go:I send the transaction
test/steps_test.go:I send the kmd-signed transaction
test/steps_test.go:I send the bogus kmd-signed transaction
test/steps_test.go:I send the multisig transaction
test/steps_test.go:the transaction should go through
test/steps_test.go:the transaction should not go through
test/steps_test.go:I sign the transaction with kmd
test/steps_test.go:the signed transaction should equal the kmd signed transaction
test/steps_test.go:I sign the multisig transaction with kmd
test/steps_test.go:the multisig transaction should equal the kmd signed multisig transaction
test/steps_test.go:I read a transaction "([^"]*)" from file "([^"]*)"
test/steps_test.go:I write the transaction to file
test/steps_test.go:the transaction should still be the same
test/steps_test.go:I do my part
test/steps_test.go:^the node should be healthy
test/steps_test.go:^I get the ledger supply
test/steps_test.go:^I get transactions by address and round
test/steps_test.go:^I get pending transactions
test/steps_test.go:^I get the suggested params
test/steps_test.go:^I get the suggested fee
test/steps_test.go:^the fee in the suggested params should equal the suggested fee
test/steps_test.go:^I create a bid
test/steps_test.go:^I encode and decode the bid
test/steps_test.go:^the bid should still be the same
test/steps_test.go:^I decode the address
test/steps_test.go:^I encode the address
test/steps_test.go:^the address should still be the same
test/steps_test.go:^I convert the private key back to a mnemonic
test/steps_test.go:^the mnemonic should still be the same as "([^"]*)"
test/steps_test.go:^mnemonic for master derivation key "([^"]*)"
test/steps_test.go:^I convert the master derivation key back to a mnemonic
test/steps_test.go:^I create the flat fee payment transaction
test/steps_test.go:^encoded multisig transaction "([^"]*)"
test/steps_test.go:^I append a signature to the multisig transaction
test/steps_test.go:^encoded multisig transactions "([^"]*)"
test/steps_test.go:^I merge the multisig transactions
test/steps_test.go:^I convert (\d+) microalgos to algos and back
test/steps_test.go:^it should still be the same amount of microalgos (\d+)
test/steps_test.go:I get account information
test/steps_test.go:I sign the bid
test/steps_test.go:key registration transaction parameters (\d+) (\d+) (\d+) "([^"]*)" "([^"]*)" "([^"]*)" (\d+) (\d+) (\d+) "([^"]*)" "([^"]*)
test/steps_test.go:I create the key registration transaction
test/steps_test.go:default V2 key registration transaction "([^"]*)"
test/steps_test.go:^I can get account information
test/steps_test.go:asset test fixture
test/steps_test.go:^default asset creation transaction with total issuance (\d+)$
test/steps_test.go:^I update the asset index$
test/steps_test.go:^I get the asset info$
test/steps_test.go:^I should be unable to get the asset info
test/steps_test.go:^the asset info should match the expected asset info$
test/steps_test.go:^I create a no-managers asset reconfigure transaction$
test/steps_test.go:^I create an asset destroy transaction$
test/steps_test.go:^I create a transaction for a second account, signalling asset acceptance$
test/steps_test.go:^I create a transaction transferring (\d+) assets from creator to a second account$
test/steps_test.go:^the creator should have (\d+) assets remaining$
test/steps_test.go:^I create a freeze transaction targeting the second account$
test/steps_test.go:^I create a transaction transferring (\d+) assets from a second account to creator$
test/steps_test.go:^I create an un-freeze transaction targeting the second account$
test/steps_test.go:^default-frozen asset creation transaction with total issuance (\d+)$
test/steps_test.go:^I create a transaction revoking (\d+) assets from a second account to creator$
test/steps_test.go:^I create a transaction transferring <amount> assets from creator to a second account$
test/steps_test.go:^base64 encoded data to sign "([^"]*)"$
test/steps_test.go:^program hash "([^"]*)"$
test/steps_test.go:^I perform tealsign$
test/steps_test.go:^the signature should be equal to "([^"]*)"$
test/steps_test.go:^base64 encoded program "([^"]*)"$
test/steps_test.go:^base64 encoded private key "([^"]*)"$
test/steps_test.go:an algod v2 client$
test/steps_test.go:^I compile a teal program "([^"]*)"$
test/steps_test.go:^it is compiled with (\d+) and "([^"]*)" and "([^"]*)"$
test/steps_test.go:^base64 decoding the response is the same as the binary "([^"]*)"$
test/steps_test.go:^I dryrun a "([^"]*)" program "([^"]*)"$
test/steps_test.go:^I get execution result "([^"]*)"$
test/steps_test.go:^I create the Method object from method signature "([^"]*)"$
test/steps_test.go:^I serialize the Method object into json$
test/steps_test.go:^the produced json should equal "([^"]*)" loaded from "([^"]*)"$
test/steps_test.go:^I create the Method object with name "([^"]*)" first argument type "([^"]*)" second argument type "([^"]*)" and return type "([^"]*)"$
test/steps_test.go:^I create the Method object with name "([^"]*)" first argument name "([^"]*)" first argument type "([^"]*)" second argument name "([^"]*)" second argument type "([^"]*)" and return type "([^"]*)"$
test/steps_test.go:^I create the Method object with name "([^"]*)" method description "([^"]*)" first argument type "([^"]*)" first argument description "([^"]*)" second argument type "([^"]*)" second argument description "([^"]*)" and return type "([^"]*)"$
test/steps_test.go:^the txn count should be (\d+)$
test/steps_test.go:^the method selector should be "([^"]*)"$
test/steps_test.go:^I create an Interface object from the Method object with name "([^"]*)" and description "([^"]*)"$
test/steps_test.go:^I serialize the Interface object into json$
test/steps_test.go:^I create a Contract object from the Method object with name "([^"]*)" and description "([^"]*)"$
test/steps_test.go:^I set the Contract\'s appID to (\d+) for the network "([^"]*)"$
test/steps_test.go:^I serialize the Contract object into json$
test/steps_test.go:^the deserialized json should equal the original Method object
test/steps_test.go:^the deserialized json should equal the original Interface object
test/steps_test.go:^the deserialized json should equal the original Contract object
test/steps_test.go:^a new AtomicTransactionComposer$
test/steps_test.go:^suggested transaction parameters fee (\d+), flat-fee "([^"]*)", first-valid (\d+), last-valid (\d+), genesis-hash "([^"]*)", genesis-id "([^"]*)"$
test/steps_test.go:^an application id (\d+)$
test/steps_test.go:^I make a transaction signer for the ([^"]*) account\.$
test/steps_test.go:^I create a new method arguments array\.$
test/steps_test.go:^I append the encoded arguments "([^"]*)" to the method arguments array\.$
test/steps_test.go:^I add a method call with the ([^"]*) account, the current application, suggested params, on complete "([^"]*)", current transaction signer, current method arguments\.$
test/steps_test.go:^I add a method call with the ([^"]*) account, the current application, suggested params, on complete "([^"]*)", current transaction signer, current method arguments, approval-program "([^"]*)", clear-program "([^"]*)"\.$
test/steps_test.go:^I add a method call with the ([^"]*) account, the current application, suggested params, on complete "([^"]*)", current transaction signer, current method arguments, approval-program "([^"]*)", clear-program "([^"]*)", global-bytes (\d+), global-ints (\d+), local-bytes (\d+), local-ints (\d+), extra-pages (\d+)\.$
test/steps_test.go:^I add a nonced method call with the ([^"]*) account, the current application, suggested params, on complete "([^"]*)", current transaction signer, current method arguments\.$
test/steps_test.go:^I add the nonce "([^"]*)"$
test/steps_test.go:^I build the transaction group with the composer\. If there is an error it is "([^"]*)"\.$
test/steps_test.go:^The composer should have a status of "([^"]*)"\.$
test/steps_test.go:^I gather signatures with the composer\.$
test/steps_test.go:^the base64 encoded signed transactions should equal "([^"]*)"$
test/steps_test.go:^I build a payment transaction with sender "([^"]*)", receiver "([^"]*)", amount (\d+), close remainder to "([^"]*)"$
test/steps_test.go:^I create a transaction with signer with the current transaction\.$
test/steps_test.go:^I append the current transaction with signer to the method arguments array\.$
test/steps_test.go:^the decoded transaction should equal the original$
test/steps_test.go:^a dryrun response file "([^"]*)" and a transaction at index "([^"]*)"$
test/steps_test.go:^calling app trace produces "([^"]*)"$
test/steps_test.go:^I append to my Method objects list in the case of a non-empty signature "([^"]*)"$
test/steps_test.go:^I create an Interface object from my Method objects list$
test/steps_test.go:^I create a Contract object from my Method objects list$
test/steps_test.go:^I get the method from the Interface by name "([^"]*)"$
test/steps_test.go:^I get the method from the Contract by name "([^"]*)"$
test/steps_test.go:^the produced method signature should equal "([^"]*)"\. If there is an error it begins with "([^"]*)"$
test/steps_test.go:^a source map json file "([^"]*)"$
test/steps_test.go:^the string composed of pc:line number equals "([^"]*)"$
test/steps_test.go:^I compile a teal program "([^"]*)" with mapping enabled$
test/steps_test.go:^the resulting source map is the same as the json "([^"]*)"$
test/steps_test.go:^getting the line associated with a pc "([^"]*)" equals "([^"]*)"$
test/steps_test.go:^getting the last pc associated with a line "([^"]*)" equals "([^"]*)"$
test/transactions_test.go:^a signing account with address "([^"]*)" and mnemonic "([^"]*)"$
test/transactions_test.go:^sign the transaction$
test/transactions_test.go:^the base64 encoded signed transaction should equal "([^"]*)"$
test/transactions_test.go:^the decoded transaction should equal the original$
test/transactions_test.go:^I build a keyreg transaction with sender "([^"]*)", nonparticipation "([^"]*)", vote first (\d+), vote last (\d+), key dilution (\d+), vote public key "([^"]*)", selection public key "([^"]*)", and state proof public key "([^"]*)"$
test/transactions_test.go:^I build an application transaction with operation "([^"]*)", application-id (\d+), sender "([^"]*)", approval-program "([^"]*)", clear-program "([^"]*)", global-bytes (\d+), global-ints (\d+), local-bytes (\d+), local-ints (\d+), app-args "([^"]*)", foreign-apps "([^"]*)", foreign-assets "([^"]*)", app-accounts "([^"]*)", fee (\d+), first-valid (\d+), last-valid (\d+), genesis-hash "([^"]*)", extra-pages (\d+)$"""

### `gosdk_df` - already Rexify'ed

In [ ]:
go_source_and_step = [s2s.split(":") for s2s in go_source2step.split("\n")]
gosdk_df = pd.DataFrame(data=[{"source": line[0], "step": line[1]} for line in go_source_and_step])
gosdk_df

In [ ]:
# gosdk_df.to_clipboard()

## 2D) Javascript

```sh
tail -n +135 tests/cucumber/steps/steps.js | grep -v '^ *//' | awk "/(Given|Then|When)/,/',/" | grep -E "\'.+\'"  | sed "s/^[^']*'\([^']*\)'.*/\1/g"
```

#### `js_steps`

In [ ]:
js_steps = """an algod client
a kmd client
an algod v2 client
wallet information
I get versions with algod
v1 should be in the versions
I get versions with kmd
I get the status
I get status after this block
I can get the block info
payment transaction parameters {int} {int} {int} {string} {string} {string} {int} {string} {string}
mnemonic for private key {string}
multisig addresses {string}
I create the payment transaction
I sign the transaction with the private key
I sign the multisig transaction with the private key
I sign the transaction with kmd
I sign the multisig transaction with kmd
the signed transaction should equal the golden {string}
the signed transaction should equal the kmd signed transaction
the multisig address should equal the golden {string}
the multisig transaction should equal the golden {string}
the multisig transaction should equal the kmd signed multisig transaction
I generate a key using kmd
I generate a key using kmd for rekeying and fund it
the key should be in the wallet
I delete the key
the key should not be in the wallet
I generate a key
I import the key
the private key should be equal to the exported private key
I get the private key
default transaction with parameters {int} {string}
default transaction with parameters {int} {string} and rekeying key
default multisig transaction with parameters {int} {string}
I import the multisig
the multisig should be in the wallet
the multisig should not be in the wallet
I export the multisig
I delete the multisig
the multisig should equal the exported multisig
the node should be healthy
I get the ledger supply
I get transactions by address and round
I get pending transactions
I get the suggested params
I get the suggested fee
the fee in the suggested params should equal the suggested fee
I create a bid
I encode and decode the bid
I sign the bid
the bid should still be the same
I decode the address
I encode the address
the address should still be the same
I convert the private key back to a mnemonic
the mnemonic should still be the same as {string}
mnemonic for master derivation key {string}
I convert the master derivation key back to a mnemonic
I create the flat fee payment transaction
encoded multisig transaction {string}
I append a signature to the multisig transaction
I merge the multisig transactions
I convert {int} microalgos to algos and back
it should still be the same amount of microalgos {int}
encoded multisig transactions {string}
I create the multisig payment transaction
I create the multisig payment transaction with zero fee
I send the transaction
I send the kmd-signed transaction
I send the multisig transaction
the transaction should go through
the transaction should not go through
I create a wallet
the wallet should exist
I get the wallet handle
I can get the master derivation key
I rename the wallet
I can still get the wallet information with the same handle
I renew the wallet handle
I release the wallet handle
the wallet handle should not work
I get account information
I can get account information
key registration transaction parameters {int} {int} {int} {string} {string} {string} {int} {int} {int} {string} {string}
I create the key registration transaction
default V2 key registration transaction {string}
asset test fixture
default asset creation transaction with total issuance {int}
default-frozen asset creation transaction with total issuance {int}
I update the asset index
I get the asset info
the asset info should match the expected asset info
I create a no-managers asset reconfigure transaction
I create an asset destroy transaction
I should be unable to get the asset info
I create a transaction for a second account, signalling asset acceptance
I create a transaction transferring {int} assets from creator to a second account
I create a transaction transferring {int} assets from a second account to creator
the creator should have {int} assets remaining
I send the bogus kmd-signed transaction
I create an un-freeze transaction targeting the second account
I create a freeze transaction targeting the second account
I create a transaction revoking {int} assets from a second account to creator
mock http responses in {string} loaded from {string}
mock http responses in {string} loaded from {string} with status {int}.
we make any {string} call to {string}.
the parsed response should equal the mock response.
expect error string to contain {string}
mock server recording request paths
expect the path used to be {string}
we expect the path used to be {string}
we make a Pending Transaction Information against txid {string} with max {int}
we make a Pending Transaction Information against txid {string} with format {string}
we make a Pending Transaction Information with max {int} and format {string}
we make a Pending Transactions By Address call against account {string} and max {int}
we make a Pending Transactions By Address call against account {string} and max {int} and format {string}
we make a Status after Block call with round {int}
we make an Account Information call against account {string} with exclude {string}
we make an Account Information call against account {string}
we make an Account Asset Information call against account {string} assetID {int}
we make an Account Application Information call against account {string} applicationID {int}
we make a Get Block call against block number {int}
we make a Get Block call against block number {int} with format {string}
we make a GetAssetByID call for assetID {int}
we make a GetApplicationByID call for applicationID {int}
we make any Pending Transaction Information call
the parsed Pending Transaction Information response should have sender {string}
we make any Pending Transactions Information call
the parsed Pending Transactions Information response should contain an array of len {int} and element number {int} should have sender {string}
we make any Send Raw Transaction call
the parsed Send Raw Transaction response should have txid {string}
we make any Pending Transactions By Address call
the parsed Pending Transactions By Address response should contain an array of len {int} and element number {int} should have sender {string}
we make any Node Status call
the parsed Node Status response should have a last round of {int}
we make any Ledger Supply call
the parsed Ledger Supply response should have totalMoney {int} onlineMoney {int} on round {int}
we make any Status After Block call
the parsed Status After Block response should have a last round of {int}
we make any Account Information call
the parsed Account Information response should have address {string}
we make any Get Block call
the parsed Get Block response should have rewards pool {string}
we make any Suggested Transaction Parameters call
the parsed Suggested Transaction Parameters response should have first round valid of {int}
we make a Lookup Asset Balances call against asset index {int} with limit {int} nextToken {string} currencyGreaterThan {int} currencyLessThan {int}
we make a Lookup Asset Balances call against asset index {int} with limit {int} afterAddress {string} currencyGreaterThan {int} currencyLessThan {int}
we make a Lookup Asset Transactions call against asset index {int} with NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {int} afterTime {int} currencyGreaterThan {int} currencyLessThan {int} address {string} addressRole {string} ExcluseCloseTo {string}
we make a Lookup Asset Transactions call against asset index {int} with NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {string} afterTime {string} currencyGreaterThan {int} currencyLessThan {int} address {string} addressRole {string} ExcluseCloseTo {string}
we make a Lookup Asset Transactions call against asset index {int} with NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {string} afterTime {string} currencyGreaterThan {int} currencyLessThan {int} address {string} addressRole {string} ExcluseCloseTo {string} RekeyTo {string}
we make a Lookup Account Transactions call against account {string} with NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {string} afterTime {string} currencyGreaterThan {int} currencyLessThan {int} assetIndex {int}
we make a Lookup Account Transactions call against account {string} with NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {string} afterTime {string} currencyGreaterThan {int} currencyLessThan {int} assetIndex {int} rekeyTo {string}
we make a Lookup Block call against round {int}
we make a Lookup Account by ID call against account {string} with round {int}
we make a Lookup Account by ID call against account {string} with exclude {string}
we make a Lookup Asset by ID call against asset index {int}
we make a LookupApplications call with {int} and {int}
we make a LookupApplicationLogsByID call with applicationID {int} limit {int} minRound {int} maxRound {int} nextToken {string} sender {string} and txID {string}
we make a Search Accounts call with assetID {int} limit {int} currencyGreaterThan {int} currencyLessThan {int} and nextToken {string}
we make a Search Accounts call with assetID {int} limit {int} currencyGreaterThan {int} currencyLessThan {int} and round {int}
we make a Search Accounts call with assetID {int} limit {int} currencyGreaterThan {int} currencyLessThan {int} round {int} and authenticating address {string}
we make a Search Accounts call with exclude {string}
we make a Search For Transactions call with account {string} NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {int} afterTime {int} currencyGreaterThan {int} currencyLessThan {int} assetIndex {int} addressRole {string} ExcluseCloseTo {string}
we make a SearchForApplications call with {int} and {int}
we make a SearchForApplications call with creator {string}
we make a Search For Transactions call with account {string} NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {string} afterTime {string} currencyGreaterThan {int} currencyLessThan {int} assetIndex {int} addressRole {string} ExcluseCloseTo {string}
we make a Search For Transactions call with account {string} NotePrefix {string} TxType {string} SigType {string} txid {string} round {int} minRound {int} maxRound {int} limit {int} beforeTime {string} afterTime {string} currencyGreaterThan {int} currencyLessThan {int} assetIndex {int} addressRole {string} ExcluseCloseTo {string} rekeyTo {string}
we make a SearchForAssets call with limit {int} creator {string} name {string} unit {string} index {int} and nextToken {string}
we make a SearchForAssets call with limit {int} creator {string} name {string} unit {string} index {int}
we make a SearchForApplications call with applicationID {int}
we make a SearchForApplications call with creator {int}
we make a LookupApplications call with applicationID {int}
we make any LookupAssetBalances call
the parsed LookupAssetBalances response should be valid on round {int}, and contain an array of len {int} and element number {int} should have address {string} amount {int} and frozen state {string}
we make a LookupAccountAssets call with accountID {string} assetID {int} includeAll {string} limit {int} next {string}
we make a LookupAccountCreatedAssets call with accountID {string} assetID {int} includeAll {string} limit {int} next {string}
we make a LookupAccountAppLocalStates call with accountID {string} applicationID {int} includeAll {string} limit {int} next {string}
we make a LookupAccountCreatedApplications call with accountID {string} applicationID {int} includeAll {string} limit {int} next {string}
we make any LookupAssetTransactions call
the parsed LookupAssetTransactions response should be valid on round {int}, and contain an array of len {int} and element number {int} should have sender {string}
we make any LookupAccountTransactions call
the parsed LookupAccountTransactions response should be valid on round {int}, and contain an array of len {int} and element number {int} should have sender {string}
we make any LookupBlock call
the parsed LookupBlock response should have previous block hash {string}
we make any LookupAccountByID call
the parsed LookupAccountByID response should have address {string}
we make any LookupAssetByID call
the parsed LookupAssetByID response should have index {int}
we make any SearchAccounts call
the parsed SearchAccounts response should be valid on round {int} and the array should be of len {int} and the element at index {int} should have address {string}
the parsed SearchAccounts response should be valid on round {int} and the array should be of len {int} and the element at index {int} should have authorizing address {string}
we make any SearchForTransactions call
the parsed SearchForTransactions response should be valid on round {int} and the array should be of len {int} and the element at index {int} should have sender {string}
the parsed SearchForTransactions response should be valid on round {int} and the array should be of len {int} and the element at index {int} should have rekey-to {string}
we make any SearchForAssets call
the parsed SearchForAssets response should be valid on round {int} and the array should be of len {int} and the element at index {int} should have asset index {int}
I add a rekeyTo field with address {string}
I add a rekeyTo field with the private key algorand address
I set the from address to {string}
we make any Dryrun call
the parsed Dryrun Response should have global delta {string} with {int}
I dryrun a {string} program {string}
I get execution result {string}
I compile a teal program {string}
it is compiled with {int} and {string} and {string}
base64 decoding the response is the same as the binary {string}
base64 encoded data to sign {string}
program hash {string}
base64 encoded program {string}
base64 encoded private key {string}
I perform tealsign
the signature should be equal to {string}
a signing account with address {string} and mnemonic {string}
suggested transaction parameters from the algod v2 client
I build a payment transaction with sender {string}, receiver {string}, amount {int}, close remainder to {string}
appID
confirmed-round
suggested transaction parameters fee {int}, flat-fee {string}, first-valid {int}, last-valid {int}, genesis-hash {string}, genesis-id {string}
I build a keyreg transaction with sender {string}, nonparticipation {string}, vote first {int}, vote last {int}, key dilution {int}, vote public key {string}, selection public key {string}, and state proof public key {string}
I build an application transaction with operation {string}, application-id {int}, sender {string}, approval-program {string}, clear-program {string}, global-bytes {int}, global-ints {int}, local-bytes {int}, local-ints {int}, app-args {string}, foreign-apps {string}, foreign-assets {string}, app-accounts {string}, fee {int}, first-valid {int}, last-valid {int}, genesis-hash {string}, extra-pages {int}
sign the transaction
the base64 encoded signed transaction should equal {string}
the decoded transaction should equal the original
an algod v2 client connected to {string} port {int} with token {string}
I create a new transient account and fund it with {int} microalgos.
I build an application transaction with the transient account, the current application, suggested params, operation {string}, approval-program {string}, clear-program {string}, global-bytes {int}, global-ints {int}, local-bytes {int}, local-ints {int}, app-args {string}, foreign-apps {string}, foreign-assets {string}, app-accounts {string}, extra-pages {int}
I sign and submit the transaction, saving the txid. If there is an error it is {string}.
I wait for the transaction to be confirmed.
I reset the array of application IDs to remember.
I remember the new application ID.
The transient account should have the created app {string} and total schema byte-slices {int} and uints {int},
 the application {string} state contains key {string} with value {string}
fee field is in txn
fee field not in txn
I create the Method object from method signature {string}
I create the Method object with name {string} first argument type {string} second argument type {string} and return type {string}
I create the Method object with name {string} first argument name {string} first argument type {string} second argument name {string} second argument type {string} and return type {string}
I create the Method object with name {string} method description {string} first argument type {string} first argument description {string} second argument type {string} second argument description {string} and return type {string}
I serialize the Method object into json
the method selector should be {string}
the txn count should be {int}
the deserialized json should equal the original Method object
I create an Interface object from the Method object with name {string} and description {string}
I serialize the Interface object into json
the deserialized json should equal the original Interface object
I create a Contract object from the Method object with name {string} and description {string}
I serialize the Contract object into json
the deserialized json should equal the original Contract object
the produced json should equal {string} loaded from {string}
a new AtomicTransactionComposer
an application id {int}
I make a transaction signer for the signing account.
I make a transaction signer for the transient account.
I create a transaction with signer with the current transaction.
I create a new method arguments array.
I append the encoded arguments {string} to the method arguments array.
I append the current transaction with signer to the method arguments array.
I add a method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments.
I add a method call with the signing account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments.
I add a method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments, approval-program {string}, clear-program {string}, global-bytes {int}, global-ints {int}, local-bytes {int}, local-ints {int}, extra-pages {int}.
I add a method call with the signing account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments, approval-program {string}, clear-program {string}, global-bytes {int}, global-ints {int}, local-bytes {int}, local-ints {int}, extra-pages {int}.
I add a method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments, approval-program {string}, clear-program {string}.
I add a method call with the signing account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments, approval-program {string}, clear-program {string}.
I add the nonce {string}
I add a nonced method call with the transient account, the current application, suggested params, on complete {string}, current transaction signer, current method arguments.
I add the current transaction with signer to the composer.
I build the transaction group with the composer. If there is an error it is {string}.
I clone the composer.
The composer should have a status of {string}.
I gather signatures with the composer.
the base64 encoded signed transactions should equal {string}
I execute the current transaction group with the composer.
The app should have returned {string}.
The app should have returned ABI types {string}.
The {int}th atomic result for randomInt\\({int}) proves correct
The {int}th atomic result for randElement\\({string}) proves correct
I can dig the {int}th atomic result with path {string} and see the value {string}
I dig into the paths {string} of the resulting atomic transaction tree I see group ids and they are all the same
The {int}th atomic result for {string} satisfies the regex {string}
a dryrun response file {string} and a transaction at index {string}
calling app trace produces {string}
I append to my Method objects list in the case of a non-empty signature {string}
I create an Interface object from my Method objects list
I create a Contract object from my Method objects list
I get the method from the Interface by name {string}
I get the method from the Contract by name {string}
the produced method signature should equal {string}. If there is an error it begins with {string}
a source map json file {string}
the string composed of pc:line number equals {string}
getting the line associated with a pc {string} equals {string}
getting the last pc associated with a line {string} equals {string}
I compile a teal program {string} with mapping enabled
the resulting source map is the same as the json {string}"""

### `jssdk_df` - Unprocessed

In [ ]:
jssdk_df = pd.DataFrame(data=[{"step": line} for line in js_steps.split("\n")])
jssdk_df

In [ ]:
# jssdk_df.to_clipboard()

### Rexify - `jssdk_rex_df`

In [ ]:
subs = {
    ')': '\)',
    '{string}': '"([^"]*)"',
    '{int}': '(\d+)',
}
jssdk_rex_df = jssdk_df.copy()
for k, v in subs.items():
    jssdk_rex_df["step"] = jssdk_rex_df.step.str.replace(k, v, regex=False)
jssdk_rex_df

In [ ]:
# jssdk_rex_df.to_clipboard()

## Step 3) simulate Cucumber reg-ex matcher

In [ ]:
def match_stepre2cucumber(step_re, cuke_df):
    matches = cuke_df[cuke_df.filled_step.str.match(step_re)]
    return None if matches.empty else matches.iloc[0].filled_step

jss = jssdk_rex_df.iloc[149]
jsm = match_stepre2cucumber(jss.step, filled_steps)

print(f"""{jss=}
{jsm=}""")

In [ ]:
def match_sdk2cucumber(sdk_rex_df, cuke_df, discard_matched=True):
    matches = sdk_rex_df.copy()
    matches["cuke"] = matches.step.apply(lambda step: match_stepre2cucumber(step, cuke_df))
    if discard_matched:
        matches = matches[pd.isna(matches.cuke)]
    return matches

### 3A) `javasdk_rex_ummatched`

In [ ]:
javasdk_rex_unmatched = match_sdk2cucumber(javasdk_rex_df, filled_steps)
javasdk_rex_unmatched

### 3B) `pysdk_rex_ummatched`

In [ ]:
pysdk_rex_unmatched = match_sdk2cucumber(pysdk_rex_df, filled_steps)
pysdk_rex_unmatched.to_clipboard()

### 3C) `gosdk_rex_ummatched`

In [ ]:
gosdk_rex_unmatched = match_sdk2cucumber(gosdk_df, filled_steps)
gosdk_rex_unmatched

### 3D) `jssdk_rex_ummatched`

In [ ]:
jssdk_rex_unmatched = match_sdk2cucumber(jssdk_rex_df, filled_steps)
jssdk_rex_unmatched

# Step 4) fuzzy match remaining SDK steps against features

In [ ]:
def fuzz_step_v_df(step, df, total_scorer=True):
    scorer = fuzz.ratio if total_scorer else fuzz.partial_ratio
    return fzp.extractOne(step, df.step, scorer=scorer)

def fuzzing_algo(samples_df, scoring_df):
    """
    samples_df - "unknown" strings to score
    scoring_df - "univierse" of known strings to search and find the "best_match"
    """
    def matcher(left, total_scorer: bool):
        matches = []
        cols = ["match", "score", "idx"]
        if total_scorer:
            cols = list(map(lambda s: f"total_{s}", cols))

        def fuzzer(row):
            return fuzz_step_v_df(row.step, scoring_df, total_scorer=total_scorer)

        msi = left.apply(fuzzer, axis=1, result_type='expand').rename(columns=dict(enumerate(cols)))
        return pd.concat([left, msi], axis=1)
    
    res = matcher(samples_df, total_scorer=True)
    res = res[res.total_score < 100]

    res = matcher(res, total_scorer=False)    
    return res

### 4A) fuzzy logic against javasdk

In [ ]:
java_remainder = fuzzing_algo(javasdk_rex_unmatched, all_steps)
java_remainder = java_remainder.sort_values(by='score')
java_remainder

In [ ]:
# java_remainder.to_clipboard()

### 4B) fuzzy logic against pysdk

In [ ]:
py_remainder = fuzzing_algo(pysdk_rex_unmatched, all_steps)
py_remainder = py_remainder.sort_values(by='score')
py_remainder

In [ ]:
# py_remaninder.to_clipboard()

### 4C) fuzzy logic against gosdk

In [ ]:
go_remainder = fuzzing_algo(gosdk_rex_unmatched, all_steps)
go_remainder = go_remainder.sort_values(by='score')
go_remainder

In [ ]:
# go_remainder.to_clipboard()

### 4D) fuzzy logic against jssdk

In [ ]:
js_remainder = fuzzing_algo(jssdk_rex_unmatched, all_steps)
js_remainder = js_remainder.sort_values(by='score')
js_remainder

In [ ]:
# js_remainder.to_clipboard()